In [1]:
#librerias 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import os 
import re 

#import model
import sys 
sys.path.append("../../1.1-Break_pruebasTranformer/")

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from model import AnomalyTransformer
import gc
from torch import optim

from torch.utils.data import DataLoader,Dataset


from sklearn.preprocessing import StandardScaler

In [2]:
BASE_DATA_DIR="../data2apply/"
data_dir=os.path.join(BASE_DATA_DIR,"pump_data")
train_dir=os.path.join(data_dir,"clean_data.csv")

In [3]:
#dataset de los datos 

class PumpData(object):
    def __init__(self,data_dir,stride=5,winsize=100,norm=True):
        self.df=pd.read_csv(data_dir,index_col=0)

        scaler=StandardScaler
        
        self.sensores_data_array=self.df.iloc[:,1:-1].to_numpy()
        if norm:
            self.sensores_data_array=StandardScaler.fit_transform(self.sensores_data_array)
        #y vamos a enventanar
        aux=[]
        for i in range(0,self.sensores_data_array.shape[0]-winsize,stride):
            aux.append(self.sensores_data_array[i:(i+winsize)])
        self.sensores_window_tensor=torch.tensor(np.array(aux))
        #self.sensores_data_tensor=torch.from_numpy(self.sensores_data_array)
        self.gt=self.df.iloc[-1].to_numpy()
        self.timestamp=self.df.iloc[1].to_numpy()


    def __len__(self):
        return self.sensores_window_tensor.shape[0]
    
    def __getitem__(self,index):
        return self.sensores_window_tensor[index]

In [4]:
#código del modelo


class AnomalyModel:
    def __init__(self, AnomalyTransformer, dataset, batch_size=16,window_size=100,enc_in=1,enc_out=1, d_model=64, n_heads=2, e_layers=2, d_ff=32,
                 dropout=0.1, activation='relu',  lambda_=1e-3,max_norm=0.1,norm_type=2):
        self.model = AnomalyTransformer(window_size, enc_in, enc_out, d_model, n_heads, e_layers, d_ff, dropout, activation, output_attention=True)
        self.model.cuda()
        self.N=e_layers
        self.model = self.xavier_initialization(self.model)
        self.dataset=dataset
        self.dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        self.lambda_ = lambda_
        self.max_norm=max_norm
        self.norm_type=norm_type

    @staticmethod
    def xavier_initialization(model):
        for module in model.modules():
            if hasattr(module, 'weight') and module.weight is not None and module.weight.dim() >= 2:
                nn.init.xavier_uniform_(module.weight)
                if hasattr(module, 'bias') and module.bias is not None:
                    nn.init.constant_(module.bias, 0)
        return model

    @staticmethod
    def my_kl_loss(p, q):
        res = p * (torch.log(p + 0.0001) - torch.log(q + 0.0001))
        return torch.mean(torch.sum(res, dim=-1), dim=1)

    @staticmethod
    def layer_association_discrepancy(Pl, Sl):
        B, Head, Height, _ = Pl.shape
        kl1 = AnomalyModel.my_kl_loss(Pl.view(B, Head, Height, -1), Sl.view(B, Head, Height, -1))
        kl2 = AnomalyModel.my_kl_loss(Sl.view(B, Head, Height, -1), Pl.view(B, Head, Height, -1))
        ad_vector = kl1 + kl2
        return ad_vector

    @staticmethod
    def association_discrepancy(P_list, S_list):
        return torch.stack([AnomalyModel.layer_association_discrepancy(j/torch.unsqueeze(torch.sum(j, dim=-1), dim=-1).repeat(1, 1, 1, 100),i) for i, j in zip(S_list,P_list)]).mean(axis=[0])

    @staticmethod
    def loss_function(x_hat, P_list, S_list, lambda_, x):
        frob_norm = ((x_hat - x)**2).sum()
        diss_norm = torch.mean(AnomalyModel.association_discrepancy(P_list, S_list))
        print(frob_norm, diss_norm)
        return (frob_norm.item(), diss_norm.item()), frob_norm - (lambda_ * diss_norm)

    @staticmethod
    def min_loss(output, P_layers, S_layers, x, lambda_):
        P_list = P_layers
        S_list = [S.detach() for S in S_layers]
        lambda_ = -lambda_
        _, loss_value = AnomalyModel.loss_function(output, P_list, S_list, lambda_, x)
        return loss_value

    @staticmethod
    def max_loss(output, P_layers, S_layers, x, lambda_):
        P_list = [P.detach() for P in P_layers]
        S_list = S_layers
        lambda_ = lambda_
        details, loss_value = AnomalyModel.loss_function(output, P_list, S_list, lambda_, x)
        return details, loss_value
    
    def cosine_lr_schedule_with_warmup(self, optimizer, epoch, initial_lr, total_epochs, warmup_epochs):
        if epoch < warmup_epochs:
            lr = initial_lr * (epoch + 1) / warmup_epochs
        else:
            adjusted_epoch = epoch - warmup_epochs
            adjusted_total_epochs = total_epochs - warmup_epochs
            lr = 0.5 * initial_lr * (1 + math.cos(math.pi * adjusted_epoch / adjusted_total_epochs))

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr


    def clip_gradients(self):
        """
        Clip gradients of the model parameters.

        Args:
            model (torch.nn.Module): The neural network model.
            max_norm (float): The maximum allowed norm for the gradients.
            norm_type (float): The type of the norm calculation (default: 2 for L2 norm).
        """
        # Recupera todos los gradientes de los parámetros del modelo
        gradients = [param.grad for param in self.model.parameters() if param.grad is not None]

        # Calcula la norma total de los gradientes
        total_norm = torch.norm(torch.stack([torch.norm(grad, self.norm_type) for grad in gradients]), self.norm_type)

        # Calcula el factor de escalado para recortar los gradientes
        clip_coef = self.max_norm / (total_norm + 1e-6)
        if clip_coef < 1:
            # Aplica el factor de escalado a los gradientes
            for grad in gradients:
                grad.mul_(clip_coef)



    def schedule_lambda(self,epoch,num_epochs,init_lambda,final_lambda):
        self.lambda_=(final_lambda-init_lambda)/num_epochs * epoch + init_lambda
        
    def train(self, num_epochs, initial_lr, warmup_epochs,init_lambda,final_lambda):
        optimizer = optim.Adam(self.model.parameters(), lr=initial_lr)

        self.model.train()
        loss_frob_diss = []

        for epoch in range(num_epochs):
            print(f"Epoch: {epoch}")
            self.schedule_lambda(epoch,num_epochs,init_lambda,final_lambda)
            for i, (inputs) in enumerate(self.dataloader):
                inputs = inputs.float().to("cuda:0")
                outputs, series, prior, _ = self.model(inputs)
                prior = [j / torch.unsqueeze(torch.sum(j, dim=-1), dim=-1).repeat(1, 1, 1, 100) for j in prior]



                loss_min = self.min_loss(outputs, prior, series, inputs, self.lambda_)
                loss_min.backward(retain_graph=True)
                optimizer.step()
                optimizer.zero_grad()

                details, loss_max = self.max_loss(outputs, prior, series, inputs, self.lambda_)
                loss_max.backward()
                optimizer.step()
                optimizer.zero_grad()

                loss_frob_diss.append(list(details))

                #loss_max.backward(retain_graph=True)
                #loss_min.backward()
                if self.max_norm is not None:
                    self.clip_gradients()
                #optimizer.step()
                #optimizer.zero_grad()

                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(self.dataloader)}], Loss: {loss_max.item():.4f}")
            if warmup_epochs is not None:
                self.cosine_lr_schedule_with_warmup(optimizer, epoch, initial_lr, num_epochs, warmup_epochs)

        print("Entrenamiento finalizado")
        return np.array(loss_frob_diss)
    
    def predict(self, data=None):
        if data is None:
            data = next(iter(self.dataloader)).float().to("cuda:0")
        else:
            data = torch.tensor(data).float()
            data = data.to("cuda:0") # .unsqueeze(0) -> esto si solo tiene un canal y no lo hemos puesto

        self.model.eval()
        with torch.no_grad():
            out, series, prior, sigmas = self.model(data)

        out = out.cpu().numpy()
        series = [s.cpu().numpy() for s in series]
        prior = [p.cpu().numpy() for p in prior]
        sigmas = [sigma.cpu().numpy() for sigma in sigmas]

        return data.cpu().detach().numpy(),out, series, prior, sigmas


    def anomaly_score(self, input=None, crit_batch_size=32):
        if input is None:
            input = self.dataset.array

        input = input.float().to("cuda:0")
        self.model.eval()
        num_batches = int(input.size(0) / crit_batch_size) + (1 if input.size(0) % crit_batch_size != 0 else 0)

        # Inicializa un tensor vacío para almacenar los resultados
        result = torch.empty(0, device="cuda:0")

        for i in range(num_batches):
            batch_start = i * crit_batch_size
            batch_end = min((i + 1) * crit_batch_size, input.size(0))

            # Procesa el lote actual
            input_batch = input[batch_start:batch_end]
            out, series, prior, sigmas = self.model(input_batch)
            ad = F.softmax(
                -AnomalyModel.association_discrepancy(prior, series), dim=0
            )

            norm = ((out - input_batch) ** 2).sum(axis=-1)

            score = torch.mul(ad, norm)

            # Añade el resultado del lote actual al tensor de resultado
            result = torch.cat((result, score), dim=0)

        return result



    def evaluate(self,data):
        #esto acepta una serie temporal, y si no se la pasamos usa la que tenia por defecto.
        #la vamos a evaluar enventanada y por partes, y luego, vamos a hacer la media de todas las lecturas del assdiss que recojamos 
        pass


In [5]:
pumpdata=PumpData(train_dir)
model_instance = AnomalyModel(AnomalyTransformer.AnomalyTransformer, pumpdata,enc_in=40,enc_out=40,max_norm=None)
#model_instance.lambda_=schedule_lambda()
# Parámetros de entrenamiento
num_epochs = 3
initial_lr = 0.001
warmup_epochs = 7

# Entrenar la instancia de la clase AnomalyModel

loss_frob_diss = model_instance.train(num_epochs, initial_lr,warmup_epochs,0.001,0.01)


Epoch: 0
tensor(1.3052e+10, device='cuda:0', grad_fn=<SumBackward0>) tensor(14.0667, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1.3052e+10, device='cuda:0', grad_fn=<SumBackward0>) tensor(14.0667, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch [1/3], Step [1/2753], Loss: 13052178432.0000
tensor(1.1533e+10, device='cuda:0', grad_fn=<SumBackward0>) tensor(14.0357, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1.1533e+10, device='cuda:0', grad_fn=<SumBackward0>) tensor(14.0357, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch [1/3], Step [2/2753], Loss: 11532836864.0000
tensor(1.3018e+10, device='cuda:0', grad_fn=<SumBackward0>) tensor(14.0423, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1.3018e+10, device='cuda:0', grad_fn=<SumBackward0>) tensor(14.0423, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch [1/3], Step [3/2753], Loss: 13017516032.0000
tensor(1.2758e+10, device='cuda:0', grad_fn=<SumBackward0>) tensor(14.0371, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1.275